# INTEGRATED USE OF MULTISOURCE REMOTE SENSING DATA FOR NATIONAL SCALE AGRICULTURAL  DROUGHT MONITORING IN KENYA
# ADM-Kenya Workshop, Kenya, May 2024
# Sentinel-2 Spectral/Vegetation Indexes Export - High-Resolution Evaportranspiration Estimation

Here, we want to load Sentinel-2 data collection. Cloud masking, generating VIs, and getting data over cropland pixels are like steps of crop condition mapping. Note that here we want to have daily products.

# Tools and Libraries

We will use the following libraries:

    openEO: A library to initiate using of openEO facility, including various dependencies for processing data collections.
    NumPy: A fundamental package for scientific computing in Python.

In [10]:
import openeo
import openeo.extra.spectral_indices as opc 
import numpy as np
import openeo.processes as eop

In [11]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


<img src="S2_Fig_1.PNG">

In [3]:
S2_2021_cloudfree = connection.load_collection(
  "SENTINEL2_L2A",
  spatial_extent={"west": 34.382, "south": 0.758, "east": 34.447, "north": 0.769},
  temporal_extent=["2021-04-01", "2021-04-30"],
  bands=["B02","B03","B04","B05","B06","B08","B11","B12","SCL"],
  max_cloud_cover=30
)

In [4]:
scl_band_2021 = S2_2021_cloudfree.band("SCL")
cloud_mask_2021 = (scl_band_2021 == 3) | (scl_band_2021 == 8) | (scl_band_2021 == 9)

In [5]:
cloud_mask_2021 = cloud_mask_2021.resample_cube_spatial(S2_2021_cloudfree)
cube_masked_2021 = S2_2021_cloudfree.mask(cloud_mask_2021)

In [6]:
# Spectral indexes:
idx_list = ["NDVI","NDMI","GNDVI","NMDI","EVI"]
S2_VIs_2021 = opc.compute_indices(cube_masked_2021, idx_list).filter_bands(idx_list)

In [7]:
# Croplands masking:
cropland_mask = connection.load_collection("ESA_WORLDCOVER_10M_2021_V2", bands=["MAP"],
                                temporal_extent = ["2021-01-01", "2022-01-01"],
                                spatial_extent={"west": 34.282, "south": 0.658, "east": 34.447, "north": 0.769}
                    )

cropland_mask = cropland_mask.band("MAP") != 40
S2_2021_input_crop = S2_VIs_2021.mask(cropland_mask.max_time())

In [8]:
job = S2_2021_input_crop.create_job(out_format="GTiff")

In [9]:
job.start_and_wait()
job.get_results()

0:00:00 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': send 'start'
0:00:32 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': created (progress 0%)
0:00:46 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': created (progress 0%)
0:01:12 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': running (progress N/A)
0:01:39 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': running (progress N/A)
0:01:57 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': running (progress N/A)
0:02:09 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': running (progress N/A)
0:02:25 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': running (progress N/A)
0:02:45 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': running (progress N/A)
0:03:09 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': running (progress N/A)
0:03:39 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': running (progress N/A)
0:04:27 Job 'j-240522232c98436ea2f13f9a5c3ba2a0': finished (progress 100%)


<JobResults for job 'j-240522232c98436ea2f13f9a5c3ba2a0'>

![image.png](attachment:0bc313a6-f49a-4875-b834-8447ba5be952.png)